# Grupo 3: Trabajo Final Integrador

Churichi Alan, Gonzalez Exequiel, Mare Rodrigo

In [1]:
import roboticstoolbox as rtb
import numpy as np
import sim
import sys
import time
from handlers import *
from kinematics import *
from utils import *
from trajectories import *
from constants import *

### Movimiento del robot utilizando el MCI calculado

In [2]:
clientID = connect(19999)

t0 = 0;
if (clientID != -1): # Mientras haya conexión con Coppelia
    joints_R1 = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    joints_R2 = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    home_R1 = {}
    home_R2 = {}
    checkpoints_move_box = [{} for i in range(CHECKPOINTS_R1_NUM)] # Checkpoints del robot 1 para mover la caja
    checkpoints_pioneer = [{} for i in range(CHECKPOINTS_PIONEER_NUM)] # Checkpoints para el robot 1 para poder ubicar las cajas en el pioneer
    checkpoints_boxes = [{} for i in range(CHECKPOINTS_BOXES_NUM)] # Checkpoints de las cajas en la cinta
    checkpoints_boxes_shelf = [{} for i in range(CHECKPOINTS_PIONEER_NUM)] # Checkpoints de las cajas en pioneer cuando está en la estantería
    checkpoints_store_box = [{} for i in range(CHECKPOINTS_R2_NUM)] # Checkpoints para mover las cajas en la estantería
    # checkpoints_shelf = [{} for i in range(CHECKPOINTS_SHELF)] # Checkpoints estantes
    
    # Se obtienen los IDs
    joint_handlers(joints_R1, 'R1', JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    joint_handlers(joints_R2, 'R2', JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_move_box, 'CheckPoint_R1', CHECKPOINTS_R1_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_pioneer, 'CheckPoint_box_pioneer', CHECKPOINTS_PIONEER_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_boxes, 'CheckPoint_box', CHECKPOINTS_BOXES_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_boxes_shelf, 'CheckPoint_box_pioneer_shelf', CHECKPOINTS_PIONEER_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_store_box, 'CheckPoint_R2', CHECKPOINTS_R2_NUM, clientID, sim.simx_opmode_blocking)
    # checkpoint_handlers(checkpoints_shelf, 'CheckPoint_shelf', CHECKPOINTS_SHELF, clientID, sim.simx_opmode_blocking)
    _, tip_R1 = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip'), sim.simx_opmode_blocking)
    _, tip_R2 = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip_2'), sim.simx_opmode_blocking)
    _, sensor_1 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_low', sim.simx_opmode_blocking)
    _, sensor_2 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_high', sim.simx_opmode_blocking)
    
    # Se setean las posiciones y orientaciones como streaming
    sim.simxGetObjectPosition(clientID, tip_R1, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip_R1, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectPosition(clientID, tip_R2, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip_R2, -1, sim.simx_opmode_streaming)
    checkpoints_set_streaming([checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes, checkpoints_boxes_shelf, checkpoints_store_box], clientID) # AGREGAR checkpoints_shelf
    sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_streaming)
    sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)

    # Se inicia la simulación
    sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)
    # sim.simxPauseSimulation(clientID, sim.simx_opmode_oneshot)
    
    time.sleep(1) # Tiempo de espera para asegurarse que los valores leidos de home sean correctos
    
    # Posición, orientación y ángulos de Home
    _, home_R1['position'] = sim.simxGetObjectPosition(clientID, tip_R1, -1, sim.simx_opmode_buffer)
    _, home_R1['orientation'] = sim.simxGetObjectOrientation(clientID, tip_R1, -1, sim.simx_opmode_buffer)
    home_R1['angles'] = inverse_kinematics(home_R1['position'], home_R1['orientation'], 'R1')
    _, home_R2['position'] = sim.simxGetObjectPosition(clientID, tip_R2, -1, sim.simx_opmode_buffer)
    _, home_R2['orientation'] = sim.simxGetObjectOrientation(clientID, tip_R2, -1, sim.simx_opmode_buffer)
    home_R2['angles'] = inverse_kinematics(home_R2['position'], home_R2['orientation'], 'R2')
    
    # Posición, orientación y ángulos de Checkpoints
    checkpoints_get_from_buffer([checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes], 'R1', clientID)
    checkpoints_get_from_buffer([checkpoints_boxes_shelf, checkpoints_store_box], 'R2', clientID) # AGREGAR checkpoints_shelf
        
    print('Calculando trayectorias...')
    
    traj = calc_trajectories(home_R1, home_R2, checkpoints_move_box, checkpoints_pioneer, checkpoints_boxes, checkpoints_boxes_shelf, checkpoints_store_box) # AGREGAR checkpoints_shelf
    sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)
    
    print('Iniciando simulación')
    
    # for i in range(BOX_COUNT):
    #     sensor_1_value = False
    #     while (sensor_1_value == False):
    #         _, sensor_1_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_buffer)
    #         _, sensor_2_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)
        
    #     sim.simxSetFloatSignal(clientID, 'beltVelocity', 0, sim.simx_opmode_oneshot)

    #     if sensor_2_value: # Caja grande
    #         for traj_point in traj['get_large_box']:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
    #         sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar

    #         for traj_point in traj['move_large_box']:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    #         for traj_point in traj['store_large_box_pioneer'][i]:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    #         sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar

    #         for traj_point in traj['back_home_large_box'][i]:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
    #     else: # Caja chica
    #         for traj_point in traj['get_small_box']:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
    #         sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar

    #         for traj_point in traj['move_small_box']:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
                
    #         for traj_point in traj['store_small_box_pioneer'][i]:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    #         sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar

    #         for traj_point in traj['back_home_small_box'][i]:
    #             time.sleep(0.05)
    #             setJointsPosition(traj_point, joints_R1, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    #     sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)   

    for traj_point in traj['prueba']:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints_R2, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    sim.simxFinish(clientID);
    print('Conexión Finalizada')



Conectado a  19999
Calculando trayectorias...
Iniciando simulación
Conexión Finalizada
